In [1]:
import pyrcn

In [2]:
import os

In [3]:
# os.listdir('/Volumes/PassportDrive')

In [3]:
# DATA_DIR = 'idrad'
DATA_DIR = '/Users/davidzhu/Desktop/IDRad/idrad'
DEFAULT_FILE = 'train/target5_001.hdf5'

# Preprocessing

In [4]:
import h5py
import numpy as np 

In [5]:
def range_doppler(data, chirps=256,
                  samples=256,
                  fft_rangesamples=2 ** 10,
                  fft_dopplersamples=2 ** 8,
                  fs=2.0e6,
                  kf=1171875.0e7,
                  min_range=0.5,
                  max_range=10):
    """
    Computes a range-doppler map for a given number of chirps and samples per chirp.
    :param data: FMCW radar data frame consisting of <chirps>x<samples>
    :param chirps: Number of chirps (Np)
    :param samples: Number of samples (N)
    :param fft_rangesamples: Number of samples for the range fft.
    :param fft_dopplersamples: Number of samples for the doppler fft.
    :param fs: Constant depending on the radar recording parameters.
    :param kf: Constant depending on the radar recording parameters.
    :param min_range: Minimum value to take into account for the range axis in the range-doppler map.
    :param max_range: Maximum value to take into account for the range axis in the range-doppler map.
    :return: Returns a 2D dimensional range-doppler map representing the reflected power over all range-doppler bins.
    """

    data = data.reshape(chirps, samples).T
    # Ignore chirp sequence number
    data = data[1:]
    Ny, Nx = data.shape  # rows (N), columns (Np)

    window = np.hanning(Ny)
    scaled = np.sum(window)
    window2d = np.tile(window, (Nx, 1)).T
    data = data * window2d

    # Calculate Range FFT
    x = np.zeros((fft_rangesamples, Nx))
    start_index = int((fft_rangesamples - Ny) / 2)
    x[start_index:start_index + Ny, :] = data
    X = np.fft.fft(x, fft_rangesamples, 0) / scaled * (2.0 / 2048)
    # Extract positive range bins
    X = X[0:fft_rangesamples // 2, :]
    # Extract range
    _freq = np.arange(fft_rangesamples // 2) / float(fft_rangesamples) * fs
    _range = _freq * 3e8 / (2 * kf)
    min_index = np.argmin(np.abs(_range - min_range))
    max_index = np.argmin(np.abs(_range - max_range))

    X = X[min_index: max_index, :]

    # Calculate Doppler FFT
    Ny, Nx = X.shape
    window = np.hanning(Nx)
    scaled = np.sum(window)
    window2d = np.tile(window, (Ny, 1))
    X = X * window2d

    rd = np.zeros((Ny, fft_dopplersamples), dtype='complex_')
    start_index = int((fft_dopplersamples - Nx) / 2)
    rd[:, start_index:start_index + Nx] = X

    range_doppler = np.fft.fft(rd, fft_dopplersamples, 1) / scaled
    range_doppler = np.fft.fftshift(range_doppler, axes=1)

    return np.abs(range_doppler)

def preprocess_file(fname): 
    with h5py.File(f'{DATA_DIR}/{fname}', 'r+') as file:
        nframes = file['radar'].shape[0]

        # Create datasets
        if not 'microdoppler' in file:
            file.create_dataset("microdoppler", (nframes, 256), dtype='float32', chunks=(1, 256))
        if not 'microdoppler_thresholded' in file:
            file.create_dataset("microdoppler_thresholded", (nframes, 256), dtype='float32', chunks=(1, 256))
        if not 'range_doppler' in file:
            file.create_dataset("range_doppler", (nframes, 380, 256), dtype='float32', chunks=True)

        
        x = file['range_doppler'][:10,:,:]
        
        #has not been preprocessed
        if np.all(x==0): 
            print('preprocessing')
        
            # Run over each radar frame
            for i in range(nframes): # only take first 1000 
                rd = range_doppler(file['radar'][i]) 
                rd = 20 * np.log10(rd)

                file['range_doppler'][i] = rd
                file['microdoppler'][i] = rd.sum(axis=0)

                rd -= np.amax(rd)
                rd[rd < -45] = -45
                file['microdoppler_thresholded'][i] = rd.sum(axis=0)

                if not i%100: 
                    print("Finished frame %d of %d." % (i + 1, nframes))
                    
def get_range_doppler(fname): 
    '''returns the range doppler'''
    preprocess_file(fname) 
    range_doppler = 0 
    
    with h5py.File(f'{DATA_DIR}/{fname}', 'r+') as file:
        # d['microdoppler'] = file['microdoppler'][:,:]
        # d['microdoppler_thresholded'] = file['microdoppler_thresholded'][:,:]
        range_doppler = file['range_doppler'][:,:,:].sum(axis=1)
    
    return range_doppler

In [6]:
from copy import deepcopy as copy 

In [7]:
hot_dict = {} 
for i in range(1,6): 
    x = np.zeros(5) 
    x[i-1] = 1 
    hot_dict[i] = x

In [8]:
def load_data(dataset_files, s): 
    dataset = [] 
    labels = []
    labels_hot = [] 
    labels_repeat = [] 
    labels_hot_repeat = [] 

    for fname in dataset_files: 
        if fname[:2] == '._':
            print(fname)
            continue 

        labels.append(int(fname[6]))
        labels_hot.append(copy(hot_dict[labels[-1]]))

        dataset.append(get_range_doppler(f'{s}/{fname}'))

        labels_repeat.append(np.repeat(labels[-1], dataset[-1].shape[0] ))
        labels_hot_repeat.append(np.repeat(labels_hot[-1][np.newaxis, :], dataset[-1].shape[0], axis=0 ))

    # dataset = np.array(dataset) 
    labels = np.array(labels)
    labels_hot = np.array(labels_hot)

    dataset = np.array(dataset, dtype = object)

    labels_repeat = np.array(labels_repeat, dtype = object)
    labels_hot_repeat = np.array(labels_hot_repeat, dtype = object)
    
    labels2, labels_hot2, dataset2, labels_repeat2, labels_hot_repeat2 = np.empty((labels.shape[0]), dtype = object), np.empty((labels.shape[0]), dtype = object), np.empty((labels.shape[0]), dtype = object), np.empty((labels.shape[0]), dtype = object), np.empty((labels.shape[0]), dtype = object)
    
    labels2[:] = list(labels)
    labels_hot2[:] = list(labels_hot)
    dataset2[:] = list(dataset)
    labels_repeat2[:] = list(labels_repeat)
    labels_hot_repeat2[:] = list(labels_hot_repeat)
    
    return labels2, labels_hot2, dataset2, labels_repeat2, labels_hot_repeat2

In [9]:
test_labels, test_labels_hot, test_dataset, test_labels_repeat, test_labels_hot_repeat = load_data(os.listdir(f'{DATA_DIR}/test'), 'test')


In [10]:
train_labels, train_labels_hot, train_dataset, train_labels_repeat, train_labels_hot_repeat = load_data(os.listdir(f'{DATA_DIR}/train'), 'train')


In [24]:
valid_labels, valid_labels_hot, valid_dataset, valid_labels_repeat, valid_labels_hot_repeat = load_data(os.listdir(f'{DATA_DIR}/valid'), 'valid')


preprocessing
Finished frame 1 of 4507.
Finished frame 101 of 4507.
Finished frame 201 of 4507.
Finished frame 301 of 4507.
Finished frame 401 of 4507.
Finished frame 501 of 4507.
Finished frame 601 of 4507.
Finished frame 701 of 4507.
Finished frame 801 of 4507.
Finished frame 901 of 4507.
Finished frame 1001 of 4507.
Finished frame 1101 of 4507.
Finished frame 1201 of 4507.
Finished frame 1301 of 4507.
Finished frame 1401 of 4507.
Finished frame 1501 of 4507.
Finished frame 1601 of 4507.
Finished frame 1701 of 4507.
Finished frame 1801 of 4507.
Finished frame 1901 of 4507.
Finished frame 2001 of 4507.
Finished frame 2101 of 4507.
Finished frame 2201 of 4507.
Finished frame 2301 of 4507.
Finished frame 2401 of 4507.
Finished frame 2501 of 4507.
Finished frame 2601 of 4507.
Finished frame 2701 of 4507.
Finished frame 2801 of 4507.
Finished frame 2901 of 4507.
Finished frame 3001 of 4507.
Finished frame 3101 of 4507.
Finished frame 3201 of 4507.
Finished frame 3301 of 4507.
Finished fra

# Layer Making

In [11]:
from pyrcn.base.blocks import InputToNode, BatchIntrinsicPlasticity, NodeToNode, HebbianNodeToNode

In [12]:
from pyrcn.echo_state_network import ESNRegressor, ESNClassifier

In [13]:
from pyrcn.base.blocks import InputToNode, PredefinedWeightsInputToNode, NodeToNode
from pyrcn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from pyrcn.model_selection import SequentialSearchCV

from sklearn.base import clone
from sklearn.metrics import make_scorer, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, ParameterGrid
from sklearn.cluster import MiniBatchKMeans
from sklearn.utils.fixes import loguniform
from sklearn.pipeline import Pipeline
from scipy.stats import uniform

In [14]:
default_rd = get_range_doppler(DEFAULT_FILE)

In [15]:
layer1_size = 500
layer2_size = 500

In [16]:
# Bilayered; i2n -> n2o, second layer has fewer (125) nodes

i2n = InputToNode(hidden_layer_size=layer1_size, input_activation="tanh")

# layer1 = i2n.fit_transform(dataset)

In [17]:
initial_params = {'hidden_layer_size': layer2_size, #1000 in 5C
                  'input_activation': 'tanh', #
                  'k_in' : 1,
                  'bias_scaling': 0.0, # usually 0 
                  'spectral_radius' : 0.0, # 
                  'reservoir_activation': 'tanh', # 2
                  'leakage': 0.05, #equation 6, 17 frames should be 1 step 
                  'bidirectional': True, #bidirectional 
                  'k_rec': layer2_size / 10, # optimal is 200 for 1000 layer size 
                  'alpha': 0.1, # figure 6, tuned with k_in
                  'random_state': 1, # 
                  'requires_sequence': True}

In [22]:
bid = False

In [18]:
l1 = Pipeline([
    ('i2n', i2n), 
              ])
n2n = Pipeline([('layer1', NodeToNode(hidden_layer_size=layer2_size, leakage=.05, bidirectional = bid)),
               ('layer2', NodeToNode(hidden_layer_size=layer2_size * 2 ** bid, leakage=.05, bidirectional = bid))])
n2o = ESNClassifier(input_to_node=l1, node_to_node=n2n)

In [19]:
n2o.fit(train_dataset, train_labels_repeat)

ESNClassifier(input_to_node=Pipeline(steps=[('i2n', InputToNode())]),
              node_to_node=Pipeline(steps=[('layer1', NodeToNode(leakage=0.05)),
                                           ('layer2',
                                            NodeToNode(leakage=0.05))]),
              regressor=IncrementalRegression(), requires_sequence=True)

In [20]:
def get_accuracy(model, data, data_labels): 
    predictions = model.predict(data)
    
    correct = []
    for i, prediction in enumerate(predictions):  

        correct.append((prediction == data_labels[i]).mean())

    correct = np.array(correct)
    
    print(correct.mean())
    
    return predictions, correct.mean()

In [21]:
train_predictions, train_acc = get_accuracy(n2o, train_dataset, train_labels)

0.8260084374556955


In [31]:
for i in range(5): 
    test_dataset[i] = test_dataset[i].astype(np.float32)

In [35]:
for i in range(5): 
    valid_dataset[i] = valid_dataset[i].astype(np.float32)

In [33]:
test_predictions, test_acc = get_accuracy(n2o, test_dataset, test_labels)

0.3132904370978478


In [36]:
valid_predictions, valid_acc = get_accuracy(n2o, valid_dataset, valid_labels)

0.38340359440869753


In [ ]:
kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}


In [ ]:
from pyrcn.base.blocks import InputToNode, PredefinedWeightsInputToNode, NodeToNode
from pyrcn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, f1_score
from pyrcn.model_selection import SequentialSearchCV
from pyrcn.echo_state_network import ESNClassifier
from pyrcn.model_selection import SequentialSearchCV

from sklearn.base import clone
from sklearn.metrics import make_scorer, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, ParameterGrid, TimeSeriesSplit
from sklearn.cluster import MiniBatchKMeans
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
from joblib import dump, load

In [ ]:
initially_fixed_params = {'hidden_layer_size': layer2_size, #1000 in 5C
                  'input_activation': 'tanh', #
                  'k_in' : 1,
                  # 'bias_scaling': 0.0, # usually 0 
                  # 'spectral_radius' : 0.0, # 
                  'reservoir_activation': 'tanh', # 2
                  # 'leakage': 0.05, #equation 6, 17 frames should be 1 step 
                  'bidirectional': True, #bidirectional 
                  # 'k_rec': layer2_size / 10, # optimal is 200 for 1000 layer size 
                  # 'alpha': 0.1, # figure 6, tuned with k_in
                  'random_state': 1, # 
                  'requires_sequence': True}

step1_esn_params = {'k_rec': uniform(10, layer2_size/2),
                    'spectral_radius': uniform(loc=0, scale=1e-3)}

step2_esn_params = {'leakage': loguniform(1e-5, 1)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': uniform(1e-5, 1e-1)}

scorer = make_scorer(accuracy_score, greater_is_better=True, needs_proba=False)
step_kwargs = {'n_iter': 50, 'random_state': 1, 'verbose': 1, 'n_jobs': -1, 'cv': TimeSeriesSplit(), 'scoring': scorer}

kwargs_step1 = {**step_kwargs}
kwargs_step2 = {**step_kwargs}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': scorer}
kwargs_step4 = {**step_kwargs}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = ESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("joblib/mse_sequential_search.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(dataset, labels_repeat)
    dump(sequential_search, "joblib/mse_sequential_search.joblib")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report


esn = sequential_search.best_estimator_

y_pred_train = esn.predict_proba(X=dataset)
cm = confusion_matrix(np.argmax(np.concatenate(labels_hot_repeat), axis=1), np.argmax(np.concatenate(y_pred_train), axis=1))
cm_display = ConfusionMatrixDisplay(cm, display_labels=[0, 1, 2, 3, 4]).plot()
print("Classification training report for estimator %s:\n%s\n" % (esn, classification_report(np.argmax(np.concatenate(labels_hot_repeat), axis=1), np.argmax(np.concatenate(y_pred_train), axis=1), digits=10)))

# y_pred_test = esn.predict(X=X_test)
# cm = confusion_matrix(np.argmax(np.concatenate(y_test), axis=1), np.argmax(np.concatenate(y_pred_test), axis=1))
# cm_display = ConfusionMatrixDisplay(cm, display_labels=[0, 1, 2]).plot()
# print("Classification training report for estimator %s:\n%s\n" % (esn, classification_report(np.argmax(np.concatenate(y_test), axis=1), np.argmax(np.concatenate(y_pred_test), axis=1), digits=10)))


In [ ]:
# model from door example



initially_fixed_params = {'hidden_layer_size': 100,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bidirectional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = ESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("RICSyN2015/sequential_search_RICSyN2015_km_large_1.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(dataset, labels_repeat)
    dump(sequential_search, "RICSyN2015/sequential_search_RICSyN2015_km_large_1.joblib")

In [ ]:
base_esn.fit(dataset, labels_repeat)

In [ ]:
predictions = base_esn.predict(dataset)

In [ ]:
correct = []
for i, prediction in enumerate(predictions):  
    
    correct.append((prediction == labels[i]).mean())

correct = np.array(correct)

In [ ]:
correct.mean()

In [ ]:
correct